# Výjimky

V programu se často vyskytují situace, kdy není definováno, jak postupovat dál. Běžný běh programu není možný (např. z důvodů, že něco chybí) a v daném místě nelze zajistit ani žádné náhradní řešení.


Příklady:

a) Funkce hledající průměr dostane prázdný seznam. Pro tento seznam není definován průměr a alternativní řešení jsou nevhodná nebo matoucí:
* vrácení hodnoty None (problém se jen odsune, s hodnotou `None` nelze dělat žádné aritmetické výpočty
* vrácení hodnoty `math.NaN` (hodnotu `Nan` nesmí vidět koncový uživatel, a obtížně se zjišťuje, kde vznikla)
* vrácení 0 (nejvíc matoucí, 0 je možným průměrem, ale nikoliv u prízdného seznamu). Jaká je průměrná tělesná teplota prázdného seznamu lidí? (0? a v jakých stupních 0°C nebo 0 K)
* nic se nevrací (v Pythonu shodné s vrácením `None`, v některých jiných jazycích nemožné)

In [1]:
max([])

ValueError: max() arg is an empty sequence

b) Funkce, která komunikuje s HTTP serverem provede operaci GET s neplatným nebo neexistujícím URL. Zde sice zdánlově existuje relativně rozumná možnost, tj. vrácení chybové hodnoty (HTTP error code), ale ty jsou podle standardu generovány HTTP serverem nikoliv klientem.

In [1]:
from requests import get

In [4]:
get("xx")

MissingSchema: Invalid URL 'xx': No schema supplied. Perhaps you meant http://xx?

c) uživatel přeruší běh programu (například během vstupu). To lze v Jupyter notebooku provést pomocí menu `Kernel|Interrupt kernel` (pří běhu skriptu pak klávesou `Ctrl+C`). Uživatel jasně naznačil, že běžné ukončení programu není možné (resp. požadované).

In [2]:
x = input()

KeyboardInterrupt: Interrupted by user

Program (přesněji řečeno běhová podpora Pythonu) zareagovala ve všech těchto případech stejně. Program je přerušen a vznikne objekt označovaný jako výjimka. Ten nese dvě základní informace:

1. důvod proč nelze pokračovat v programu
2. místo, kde byl program přerušen

Standardní reakcí na vznik výjimky (a tím i dočasné) přerušení programu je bezprostřední dokončení programu a výpis informací o výjimce (krátký anglický text + místo vzniku výjimky). 

Jednou z možných interpretací výjimky je vzdát **vzdání se odpovědnosti**. Když už nějaký kód neví jak dál, tak se může vzdát odpovědnosti  a nechat řešení na jiné části kódu počítaje v to i kód, který program rozumně ukončí (rozumně = například s hlášením, co a kde se stalo)

> **Úkol**: Uveďte i další případy, kdy program končí výjimkou.

## Vyhození výjimky

Výjimka však nemusí vzniknout jen externím přičiněním (tj. přerušením programu uživatelem resp. operačním systémem) nebo uvnitř knihovních metod a funkcí (jako tomu bylo u funkce `max` či `requests.get`). Může to udělat o programátor. pokud se dostane do situace. kdy neexistuje žádné obecně použitelné řešení a musí se tak vzdát odpovědnosti.

Děje se tak mechanismem označovaným jako **vyhození výjimky**. Příkaz `raise` pozastaví program a vytvoří objket výjimky, která nese informaci o příčině.

Ukažme si příklad. Následující funkce má za úkol najít průsečík dvou přímek zadaných v obecném tvaru $ax+by+c=0$. Funkce by měla vracet jeden bod. V případě, že jsou přímky rovnoběžné, není návratová hodnota definována (protože takový bod neexistuje resp. existuje nekonečně mnoho bodů). Funkce se proto zbaví odpovědnosti tím, že vyhodí výjimku.

In [6]:
def intersection(p, q):
    pa,pb,pc = p # dekonstrukce hodnot z n-tic resp. seznamů
    qa,qb,qc = q
    det = pa * qb - pb * qa
    if pa * qb - pb * qa == 0:  # normálové vektory jsou závislé (determinant = 0)
        raise Exception("Parallel lines") # vyhození výjimky
    return (pc * qb - pb * qc)/det, (pa * qc - pc * qa)/det # Crammerovo pravidlo
    

In [4]:
intersection((1,1,0), (0,1,3)) # průsečík existuje

(-3.0, 3.0)

In [7]:
intersection((2,3,4), (4,6,1))

Exception: Parallel lines

Při návrhu metody by bylo lze uvažovat i o jiném řešení nedefinovaného případu. Funkce by v tomto případě mohla vrátit nějakou speciální hodnotu, v tomto případě například `None`. To je však méně přehledné (`None` v tomto případě representuje, jak neexistenci bodu tak existenci několika možných bodů). Navíc zátěž testování (které je nutné, neboť `None` objekt nelze využít jako běžnou representaci 2D bodu) by byla na kódu, který danou funkci využívá.

> **Základní pravidlo**: Jakýkoliv program by měl buď fungovovat podle specifikace (tj. poskytovat správné výsledky nebo u interaktivních funčnost) nebo být předčasně ukončen výjimkou.

Objekt výjimky v našem případě instancí třídy `Exception` (v rámci výrazu za příkazem `raise` se volá konstruktor této třídy). Obecně však mohou být objekty výjimek i instancemi specializovanějších tříd.
Přehhled vestavěných tříd výjimek najdete na stránkách https://docs.python.org/3/library/exceptions.html#base-classes resp. https://docs.python.org/3/library/exceptions.html#concrete-exceptions (o něco konkrétnější výjimky).

> **Úkol**: Pokuste se najít konkrétnější výjimku pro náš příklad s průsečíkem přímek (a použijte ji v pro
gramu).

Nejvhodnější specifičtější třída výjimek pro náš případ je výjimka `ValueError`, která se využívá v případě nepodporovaných hodnot vstupních parametrů. 

In [7]:
def intersection2(p, q):
    pa,pb,pc = p # dekonstrukce hodnot z n-tic resp. seznamů
    qa,qb,qc = q
    det = pa * qb - pb * qa
    if pa * qb - pb * qa == 0:  # normálové vektory jsou závislé (determinant = 0)
        raise ValueError("Intersection is not defined for parallel lines") # vyhození výjimky
    return (pc * qb - pb * qc)/det, (pa * qc - pc * qa)/det # Crammerovo pravidlo

## Aserce (testovací tvrzení)

Mechanismus výjimek se využívá i v případě, tzv. asercí, které primárně kontrolují sémantické chyby v programu. Aserce je test, který ověřuje, že jsou splněny všechny předpoklady pro úspěšné pokračování programu (tj. jinak řečeno zda se program nachází v definovaném stavu), Pokud je podmínka splněna (vše je OK), program pokračuje bez jakéhokoliv ovlivnění, pokud splněna není je přerušen a je vyhozena výjimka `AssertionError`.

In [2]:
a = 2
b = 2
assert a != b, "Hodnoty a, b nesmí být stejné"
x = 2 / (a-b)

AssertionError: Hodnoty a, b nesmí být stejné

Ne zcela intuitivní je u asercí skutečnost, že podmínka definuje požadovaný (tj. bezchybný stav a nepovinná zpráva naopak popisuje stav chybový (tj. je-li splněna negace podmínky)!

Použití asercí se významně překrývá s oblastí použití výjimek. Mezi základní rozlišovací charakteristiky patří:
* aserce signalizují interní chyby nikoliv chybné vstupy či jiné vnější chyby (nedostatek prostředků). Tj. v modelu předpokládáme, že hodnoty proměnnách `a`, `b` jsou vždy různé (což může být zajištěno již ve stupní rutině např. znemožněním zadání stetjných hodnot v GUI formuláři, či testováním přípustných hodnot při čtení datových souborů), pak je namístě *aserce* (chyba vznikne např. špatným výpočtem, resp. použitím špatné funkce). V opačném případě je namístě vyvolání výjimky
* u asercí se předpokládá, že vždy povedou k předčasnému konci programu (jiné řešení znamená, že akceptujeme či se snažíme zakrýt nějakou berličkou chybu programu)

Při běhu programu se chování asercí od běžných výjimek liší v tom, že v tzv. release nasazení (tj. ve finálním nastavení u zákazníka) neuplatňují (tj. se nevznikají výjimky a nedochází ani k testování testovací podmínky), jinak řečeno aserce v release nasazení program nezpomalují.  

V Pythonu jsou aserce vypínány v případě, že vestavěná proměnná `__debug__` nabývá hodnoty `False`, což se děje jen v případě, že překladač spustíte s přepínačem `-O` (tím se zapne tzv, optimalizace kódu).

> *Upozornění*: vypnutím asercí se samozřejmě neodstraní příčina chyby. Program musí být nejdříve důkladně otestován a teprve pak se spouštěn v optimalizovaném `release` režimu.

Aserci můžete použít i v našem případě s průsečíkem přímek. Je však nutné zajistit, že odpovědnost za zajištění nerovnoběžnosti přímek bude kódu volajícím danou funkci. To lze zajistit například tím, že tento požadavek uvedeme do dokumentace k dané funkci. Dokumentace se uvádí jako řetězec (často víceřádkový) za hlavičkou funkce.

In [3]:
def intersection3(p, q):
    """
    Průsečík dvou přímek. Přímky přímky nesmí být rovnoběžné či identické.
    Args:
        p: trojice koeficientů (a,b,c) z obecného tvaru první přímky  ax + by + c = 0
        q: trojice koeficientů (a,b,c) z obecného tvaru druhé přímky  ax + by + c = 0
    """
    pa,pb,pc = p # dekonstrukce hodnot z n-tic resp. seznamů
    qa,qb,qc = q
    det = pa * qb - pb * qa
    assert pa * qb - pb * qa != 0, "Paralllel lines are not supported"
    return (pc * qb - pb * qc)/det, (pa * qc - pc * qa)/det # Crammerovo pravidlo 

In [4]:
help(intersection3) # takto lze zobrazit dokumentaci (zobrazuje se i v růůzných IDE)

Help on function intersection3 in module __main__:

intersection3(p, q)
    Průsečík dvou přímek. Přímky přímky nesmí být rovnoběžné či identické.
    Args:
        p: trojice koeficientů (a,b,c) z obecného tvaru první přímky  ax + by + c = 0
        q: trojice koeficientů (a,b,c) z obecného tvaru druhé přímky  ax + by + c = 0



In [16]:
intersection3((1,0,1), (2,0,2))  # byli jste varováni

AssertionError: Paralllel lines are not supported

> **Úkol**:  Konkrétní formát dokumentačníjo řetězce Python nepředepisuje. V praxi existuje několik formátů podporovaných různými nástroji (generátory dokumentace, IDE). Z přehledu na stránkách [datacampu](https://www.datacamp.com/community/tutorials/docstrings-python) (autor Aditya Sharma) určete jaký formát byl použit v příkladě (resp. proč?).

Byl použit formát *Google Style Docstrings*, který je nejjednodušší a nejstručnější z běžně používaných formátů.

## Správce kontextu 

Při vzniku výjimky zanikají všechny kontexty, ve kterých došlo jejímu vyhození. Zanikají proměnné, uvolňují se objekty, apod. Jinak řečeno výjimky zajistí, že se i po předčasném skončení funkcí, či celých programů zanechá čistý stůl.
To však platí jen v případě zdrojů umístěných v operační paměti. Pokud jsou v daném kontextu drženy nějaké zdroje operačního systému (otevřené soubory, dočasné soubory, síťová spojení, bitmapy v GPU, apod.) tak se neuvolní!

Podívejme se na následující příklad. Je to funkce, která otevře konfigurační soubor s názvem `network.ini`, která obsahuje jediný řádek, který definuje hodnotu konfigurační volby `network`, která může nabývat hodnoty `yes` nebo `no`.

Nejdříve vytvoříme příslušný konfigurační soubor.

In [29]:
%%writefile network.ini
network=no

Overwriting network.ini


In [30]:
def network_configuration():
    import re
    f = open("network.ini")
    line = f.read()
    match = re.match("network=(yes|no)", line)
    if not match:
        raise Exception("Invalid configuration file")
    f.close() # nesmíme zapomenout zavřít soubor
    return match.group(1) == "yes"

network_configuration()

False

Nyní vytvoříme soubor s chybnou syntaxí (hodnota `false` není podporována).

In [31]:
%%writefile network.ini
network=false

Overwriting network.ini


In [32]:
network_configuration()

Exception: Invalid configuration file

Výsledek není překvapivý. Je vyolána výjimka, neboť obsah konfiguračního souboru není v našem modelu interpretovatelný.
Je zde však i jeden nepříjemný důsledek. Soubor `network.ini` je otevřen, ale není uzavřen! A to navzdory tomu, že je explicitně uzavírán na řádku pod vyhozením výjimky.

Důvod je zřejmý. Po vyhození výjimky se se zbytek funkce neprovede (výjimka opustí funkci) a neprovede se tedy ani volání metody `close`. V běžných programech se sice soubor uzavře po ukončení programu, ale k tomu nemusí dojít, neboť výjimka může být tzv. zachycena. Děje se tak i v případě vyvolání funkce v Jupyter notebooku. V tomto případě nejen, že nedošlo k uzavření souboru, ale soubor už ani nelze uzavřít (lokální proměnná totiž přestává existovat a my nemáme žádný odkaz na objekt zpřístupňující otevřený soubor). 

Neuzavření souboru sice většinou nebrání dalšímu běhu, snižuje se však množství volných prostředků. Po určoté době prostředky dojdou. V případě Linuxu i dalších operačních systémů je množství simultánně otevřených souborů na proces relativně vysoké, ale i tak není nekonečné.

> **Úkol**: Zjistěte jaké maximální simultánně běžících procesů může mít váš Jupyter notebook (v rámci vaší instance operačního systému).

Řešení pro Linux a unixové systémy (MS Windows nepoužívám):

In [8]:
!ulimit -n

4096


Základním řešením tohoto problému je využití tzv. správců kontextů. Objekty, které fungují jako správci kontextů podporují dvě metody. Metoda `__enter__` je volána při vstupu do určitého úseku kódu (tj. například do úseku kódu využívajícího otevřený soubor), metoda `__exit__` při výstupu, ať už je úsek kódu opuštěn doasažením konce, výskokem (např. příkazem `return`) nebo vyhozením výjimky.

Příslušný úsek kódu je definován příkazem `with`, za nímž následuje odsazený blok. Na začátku bloku se typicky vytváří objekt funngující (mimo jiné) jako kontextový manager a je volána jeho metoda `__enter__`. Na konci bloku se volá metoda `__exit__`, která uvolní prostředky svázané s objektem (samotná objekt nicméně stále existuje).

In [12]:
with open("network.ini", "rt") as f:
    print(f.read())

network=false



Za klíčovým slovem `with` je výraz, jehož vyhodnocením vznikne objekt proudu, který se zároveň stane správcem kontextu, je na něm volána metoda `__enter__` a je opatřen jménem `f`. Uvnitř odsazeného bloku (zde je to jediný řádek) lze využívat proměnnou, objekt, který odkazuje, i otevřený soubor, který je vlastněn objektem (tj. ze souboru lze číst).

Po ukončení existuje jak proměnná (zde `f`, tak objekt souboru) tak objekt proudu, je však uzavřen soubor, který je s ním spjat (uvnitř volání metody `exit`). Ten již proto zbytečně nealokuje prostředky. Nelze jej však využít pro další práci.

In [13]:
f.read()

ValueError: I/O operation on closed file.

> **Úkol**: Vyzkoušejte, že k uzavření souboru dojde i v případě, že uvnitř bloku po `with` dojde ke vzniku výjimky. Implentujte správnou verzi funkce `network_configuration`.

In [15]:
def network_configuration():
    import re
    with open("network.ini") as f:
        line = f.read()
        match = re.match("network=(yes|no)", line)
        if not match:
            raise Exception("Invalid configuration file") # opuštění with bloku výjimkou
        return match.group(1) == "yes"                    # opuštění with bloku ukončením funkce (výskokem)

network_configuration()

Exception: Invalid configuration file

> **Úkol**: Porovnejte použití správce kontextů mezi bežnými soubory a soubory dočasnými (viz standardní modul `tempfile`). Vyzkoušejte na příkladě. (můžete použít příklad z dokumentace).

Základní rozdíl je v tom, že dočasný soubor je po ukončení svého kontextu (= blok po `with`) smazán. Všiměte si také, že nemusí být smazán.